In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.widget-label { display: contents !important; }</style>"))
display(HTML("<style>.slider-container { margin: 12px !important; }</style>"))
display(HTML("<style>.jupyter-widgets { overflow: auto !important; }</style>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [3]:
def get_relative_isotherms(v_range, T_range):
    
    """
    
        This function calculates the p(v, T) plane
        from given a and b parameters of 
        the Van der Waals equation of state for real gases.

    Args:
        a: Term related with the attraction between particles in
        J m^3/ mol^2.\n
        b: Term related with the volume that is occupied by one 
        mole of the molecules in 10^-3 m^3/mol.\n
        T_range: Tuple containing maximum and minimum values of
        temperature to be plotted. Temperatures must be expressed
        in terms of the critical temperature.\n
        v_range: Tuple containing maximum and minimum values of
        volumen to be plotted. Volumens must be expressed
        in exponents of the critical volumen (v in 10^v_range).\n
        
    Returns:
        isotherms: a dictionary containing the values of v and the
        pressures of the isotherms in the following form:\n
            isotherms['v'] = np.array containing the values of v
            in a logarithmic scale.\n
            isotherms['value of the isotherm'] = np.array containing
            the values of p.\
    """

    isotherms = []

    for T in T_range:
        p_R = []
        for v in v_range:
            val = (8.0/3.0*T/(v - 1.0/3.0) - 3.0/v**2)
            p_R = np.append(p_R, val)

        isotherms.append(p_R)

    return isotherms

In [4]:
def update_tracer(change):
    
    tracer_117_003.visible = True
    
    if p_slider.value == p_slider.max:
        p_slider.disabled = True
    
    i = change.get('owner').value
    v = dense_v_values_inverted[T_slider.index][i]
    p = dense_p_values_inverted[T_slider.index][i]

    tracer_117_003.x, tracer_117_003.y = [p], [v]

    area_117_003.x = np.append(area_117_003.x, p)
    area_117_003.y = np.append(area_117_003.y, v)
        
    lines_117_004.x = np.append(lines_117_004.x, p)
    lines_117_004.y = np.append(lines_117_004.y, mu[T_slider.index][i])

In [5]:
def get_chemical_potential(p_values, v_values):
    
    #p: ND array; v: ND array
    
    mu = []
    
    
    for i in range(len(v_values)):

        v = v_values[i]
        p = p_values[i]
        
        pot = [10.0] #starting random value
        
        l = np.size(p)
        
        for j in range(1, l):
            pot.append(pot[j-1] + v[j]*(p[j] - p[j-1]))
        
        mu.append(pot)
        
    return mu

In [6]:
def restart_chemical_potential(a):
    
    lines_117_004.x, lines_117_004.y = [], []
    area_117_003.x, area_117_003.y = [], []
    
    p_slider.value = p_slider.min
    p_slider.disabled = False

In [7]:
def change_tenperature(change):

    p_slider.max = len(mu[T_slider.index])-1
    restart_chemical_potential(None)
    
    obj = change.owner
    
    #p_values = get_relative_isotherms(v_values, [obj.value])
    
    lines_117_001.x, lines_117_001.y = v_values, p_values[T_slider.index]
    lines_117_002.x, lines_117_002.y = p_values[T_slider.index], v_values
    
    tracer_117_003.x, tracer_117_003.y = [lines_117_002.x[0]], [lines_117_002.y[0]]
    
    axis_y_004.scale =  bqs.LinearScale(min = min(mu[T_slider.index]), max = max(mu[T_slider.index]))
    lines_117_004.scales = {'x': scale_x_004, 'y': axis_y_004.scale}
    
    
    axis_y_004.tick_values = np.linspace(min(mu[T_slider.index]), max(mu[T_slider.index]), 4)
    #dense_p_values = get_relative_isotherms(dense_v_values, [obj.value])[0]
    #
    #i_in_range = np.where(dense_p_values < 2.0)
#
    #dense_v_values_filtered = np.take(dense_v_values, i_in_range)[0]
    #dense_p_values_filtered = np.take(dense_p_values, i_in_range)[0]
    #
    #dense_v_values_inverted = np.flip(dense_v_values_filtered)
    #dense_p_values_inverted = np.flip(dense_p_values_filtered)
    #
    #dense_p_values_rounded = np.round(np.flip(dense_p_values_filtered), 3)
    #
    #p_slider.options, p_slider.value = dense_p_values_rounded, dense_p_values_rounded[0]
#
    #new_mu = get_chemical_potential(dense_p_values_inverted, dense_v_values_inverted)
    #print(len(mu))
    #for i in range(len(mu)):
    #    if i < len(new_mu):
    #        mu[i] = new_mu[i]
    #    else:
    #        try:
    #            mu.pop(i)
    #        except:
    #            print(i)

In [8]:
def show_all_potentials(change):
    
    max_limit = 0.0
    min_limit = 100.0
    
    for pot in mu:
        max_pot = max(pot)
        min_pot = min(pot)
        
        if max_pot > max_limit:
            max_limit = max_pot
            
        if min_pot < min_limit:
            min_limit = min_pot
            
    axis_y_004.scale =  bqs.LinearScale(min = min_limit, max = max_limit)

    lines_117_004.scales = {'x': scale_x_004, 'y': axis_y_004.scale}
    axis_y_004.tick_values = np.linspace(0.0, max_limit, 4)

   
    lines_117_004.x, lines_117_004.y = [p.tolist() for p in dense_p_values_inverted], [m.tolist() for m in mu]

In [9]:
def update_text(change):
    
    obj = change.owner
    i = obj.value
    
    p_text.value = '<p>' + str(dense_p_values_rounded[T_slider.index][i])+ '</p>'

In [10]:
v_values = np.linspace(0.4, 5.0, 500)
T_values = [0.85, 0.9, 1.0, 1.2]
p_values = get_relative_isotherms(v_values, T_values)

In [11]:
################################
######1ST BLOCK#################
################################

block_1 = widgets.VBox([], layout=widgets.Layout(align_items='center'))

scale_x = bqs.LinearScale(min = min(v_values), max = max(v_values))
scale_y = bqs.LinearScale(min = 0.0, max = 2.0)

axis_x_001 = bqa.Axis(scale=scale_x,
                    tick_format='.2f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    tick_values = np.linspace(min(v_values), max(v_values), 5),
                    #num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='v',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_001 = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(0, 2.0, 4),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_117_001 = Figure(
      title='',
       marks=[],
       axes=[axis_x_001, axis_y_001],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=50, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )

lines_117_001 = bqm.Lines(
       x = v_values, 
       y = p_values[0], 
       scales = {'x': scale_x, 'y': scale_y}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = ['red'],
       #labels = T_values_str,
       )

fig_117_001.marks = [lines_117_001]

axis_x_002 = bqa.Axis(scale=scale_y,
                    tick_format='.1f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    tick_values = np.linspace(0, 2.0, 4),
                    #num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='p',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_002 = bqa.Axis(
                scale=scale_x,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(min(v_values), max(v_values), 5),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='v',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_117_002 = Figure(
      title='',
       marks=[],
       axes=[axis_x_002, axis_y_002],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=50, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )

lines_117_002 = bqm.Lines(
       x = p_values, 
       y = v_values, 
       scales = {'x': scale_y, 'y': scale_x}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = ['red'],
       #labels = T_values_str,
       )

fig_117_002.marks = [lines_117_002]

block_1.children = [widgets.HBox([fig_117_001, fig_117_002], layout=widgets.Layout(width='100%'))]

In [12]:
################################
######2ND BLOCK##############
################################

block_2 = widgets.VBox([], layout=widgets.Layout(align_items='center'))

T_slider = widgets.SelectionSlider(
    options= T_values,
    value=T_values[0],
    description=r'\( T \)',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    layout = widgets.Layout(width = '300px', height = 'auto', margin='0 0 0 50px')
)

T_slider.observe(change_tenperature, 'value')

block_2.children = [T_slider]

In [13]:
################################
######3RD BLOCK##############
################################

block_3 = widgets.HBox([], layout=widgets.Layout(align_items='center'))

dense_v_values = np.linspace(min(v_values), max(v_values), 10000)
dense_p_values = get_relative_isotherms(dense_v_values, T_values)

dense_v_values_filtered = []#np.array([])
dense_p_values_filtered = []#np.array([])
#
dense_v_values_inverted = []#np.array([])
dense_p_values_inverted = []#np.array([])

dense_p_values_rounded = []#np.array([])

for i in range(len(T_values)):    

    i_in_range, = np.where(dense_p_values[i] < 2.0)

    dense_v_values_filtered.append(np.take(dense_v_values, i_in_range))
    dense_p_values_filtered.append(np.take(dense_p_values[i], i_in_range))
    
    dense_v_values_inverted.append(np.flip(dense_v_values_filtered[i]))
    dense_p_values_inverted.append(np.flip(dense_p_values_filtered[i]))
    
    dense_p_values_rounded.append(np.round(np.flip(dense_p_values_filtered[i]), 3))
    #dense_v_values_filtered = np.append(dense_v_values_filtered, np.take(dense_v_values, i_in_range))
    #dense_p_values_filtered = np.append(dense_p_values_filtered, np.take(dense_p_values[i], i_in_range))
#
    #dense_v_values_inverted = np.append(dense_v_values_inverted, np.flip(dense_v_values_filtered))
    #dense_p_values_inverted = np.append(dense_p_values_inverted, np.flip(dense_p_values_filtered[i]))
#
    #dense_p_values_rounded = np.append(dense_p_values_rounded, np.round(np.flip(dense_p_values_filtered[i]), 3))

fig_117_003 = Figure(
      title='',
       marks=[],
       axes=[axis_x_002, axis_y_002],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=50, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )

area_117_003 = bqm.Lines(
       x = [], 
       y = [], 
       scales = {'x': scale_y, 'y': scale_x}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = ['#39362d'],
       fill = 'bottom',
       fill_colors = ['#ffd429'],
       fill_opacities = [0.4]
       #labels = T_values_str,
       )

tracer_117_003 = bqm.Scatter(
        name = '',
        x = [0.0],
        y = [0.0],
        scales = {'x': scale_y, 'y': scale_x}, 
        opacity = [1.0, 0.0],
        visible = False,
        colors = ['#2807a3'],
    )

fig_117_003.marks = [lines_117_002,
                     area_117_003,
                     tracer_117_003]

mu = get_chemical_potential(dense_p_values_inverted, dense_v_values_inverted)

for i in range(len(mu)):
    mu[i] = np.array(mu[i]) - 10.0*T_slider.options[i]

p_text = widgets.HTML(
    value="<p>" + str(dense_p_values_rounded[T_slider.index][i]) + "</p>",
    layout = widgets.Layout(height='auto', margin='10px 0 0 10px')
)

p_slider = widgets.IntSlider(
    min=0,
    max=len(mu[T_slider.index])-1,
    value=0,
    description=r'\( v \)',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    layout = widgets.Layout(width = '60%', height='auto', margin='0 0 0 50px')
)

p_slider.observe(update_tracer, 'value')
p_slider.observe(update_text, 'value')


#play = widgets.Play(
#    interval=1,
#    value=0,
#    min=0,
#    max=len(p_slider.options),
#    step=1,
#    description="Press play",
#    disabled=False
#)
#widgets.jslink((play, 'value'), (p_slider, 'index'));



scale_x_004 = bqs.LinearScale(min = 0.0, max = 2.0)
scale_y_004 = bqs.LinearScale(min = min(mu[T_slider.index]), max = max(mu[T_slider.index]))

axis_x_004 = bqa.Axis(scale=scale_x_004,
                    tick_format='.2f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    tick_values = np.linspace(min(dense_p_values_inverted[T_slider.index]), max(dense_p_values_inverted[T_slider.index]), 5),
                    #num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='p',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_004 = bqa.Axis(
                scale=scale_y_004,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = 0,
                #tick_values = np.linspace(min(mu[T_slider.index]), max(mu[T_slider.index]), 4),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='mu',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_117_004 = Figure(
      title='',
       marks=[],
       axes=[axis_x_004, axis_y_004],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=50, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )

lines_117_004 = bqm.Lines(
       x = [], 
       y = [], 
       scales = {'x': scale_x_004, 'y': scale_y_004}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = ['red'],
       #labels = T_values_str,
       )

fig_117_004.marks = [lines_117_004]

restart_button = widgets.Button(
        description='Clean figure',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip="",
        layout = widgets.Layout(height='auto')
    )

restart_button.on_click(restart_chemical_potential)


show_all_button = widgets.Button(
        description='Show all potentials',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip="",
        layout = widgets.Layout(height='auto')
    )

show_all_button.on_click(show_all_potentials)

block_3.children = [widgets.VBox([fig_117_003, widgets.HBox([p_slider, p_text])], layout=widgets.Layout(height='auto')),
                         widgets.VBox([fig_117_004, restart_button, show_all_button], layout=widgets.Layout(align_items='center'))]

In [14]:
def get_roots(p, T):
    
    roots = np.roots([1.0, - 1.0/3.0*(1.0 + 8.0*T/p), 3.0/p, -1.0/p])
    roots_in_range = []
    
    for root in roots:
        if np.isreal(root):
            root = np.real(root)
            if root > 0:
                roots_in_range.append(root)
    #print(roots_in_range) 
    roots_in_range.sort()
    
    return roots_in_range

In [15]:
def p_undefined_integral(p_0, v_0, T):
    
    return 8.0/3.0 * T *np.log(v_0 - 1.0/3.0) + 3.0/v_0 - p_0*v_0

In [16]:
def defined_integral(p_0, v_range, T):
    v_0, v_1 = v_range[0], v_range[1]
    return p_undefined_integral(p_0, v_1, T) - p_undefined_integral(p_0, v_0, T)

In [17]:
def find_real_fixed_T(p_range, T):
    
    eps = 1e-3 #1.0/p_range.size
    
    for p in p_range:
        
        roots = get_roots(p, T)
        if len(roots) == 3:
            v_range = [roots[0], roots[2]]
            area = defined_integral(p, v_range, T)
            if abs(area) < eps:
                return p, v_range

    return None

In [18]:
################################
######4TH BLOCK#################
################################

block_4 = widgets.HBox([], layout=widgets.Layout(align_items='center'))

trans_pressures = []
trans_pressures_repeated = []
trans_volumens = []
trans_potentials = []

for i in range(len(T_values)):
    T = T_values[i]
    p = p_values[i]
    
    if T < 1.0:
        p_trans, v_range = find_real_fixed_T(p, T)

        if p_trans is not None:
            trans_pressures = np.append(trans_pressures, p_trans)
            trans_pressures_repeated = np.append(trans_pressures_repeated, [p_trans, p_trans])
            trans_volumens = np.append(trans_volumens, v_range)
            
        p_v = dense_p_values_inverted[i]
        index, = np.where(abs(p_v - p_trans) < 1e-4)
        if len(index) != 0:
            trans_potentials = np.append(trans_potentials, mu[i][index[0]])
                
    elif T == 1.0:
        
            trans_pressures = np.append(trans_pressures, [1.0])
            trans_pressures_repeated = np.append(trans_pressures_repeated, [1.0])
            trans_volumens = np.append(trans_volumens, 1.0)
            
            p_v = dense_p_values_inverted[i]
            index, = np.where(abs(p_v - 1.0) < 1e-10)
            if len(index) != 0:
                trans_potentials = np.append(trans_potentials, mu[i][index[0]])
            
lines_117_005 = bqm.Lines(
       x = v_values,
       y = p_values, 
       scales = {'x': scale_x, 'y': scale_y}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = ['red'],
       #labels = T_values_str,
       )

scatter_117_005 = bqm.Scatter(
    name = '',
    x = trans_volumens,
    y = trans_pressures_repeated,
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['black'],
)   

fig_117_005 = Figure(
      title='',
       marks=[lines_117_005, scatter_117_005],
       axes=[axis_x_001, axis_y_001],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=50, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )



max_limit = 0.0
min_limit = 100.0

for pot in mu:
    max_pot = max(pot)
    min_pot = min(pot)

    if max_pot > max_limit:
        max_limit = max_pot

    if min_pot < min_limit:
        min_limit = min_pot
      
        
scale_x_006 = bqs.LinearScale(min = 0.0, max = 2.0)
scale_y_006 = bqs.LinearScale(min = min_limit, max = max_limit)

axis_x_006 = bqa.Axis(scale=scale_x_006,
                    tick_format='.2f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    tick_values = np.linspace(0.0, 2.0, 5),
                    #num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='p',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_006 = bqa.Axis(
                scale=scale_y_006,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = 0,
                #tick_values = np.linspace(min(mu[T_slider.index]), max(mu[T_slider.index]), 4),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='mu',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

lines_117_006 = bqm.Lines(
       x = [p.tolist() for p in dense_p_values_inverted],
       y = [m.tolist() for m in mu], 
       scales = {'x': scale_x_006, 'y': scale_y_006}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = ['red'],
       #labels = T_values_str,
       )

scatter_117_006 = bqm.Scatter(
    name = '',
    x = trans_pressures,
    y = trans_potentials,
    scales = {'x': scale_x_006, 'y': scale_y_006}, 
    #opacities = [1.0],
    visible = True,
    colors = ['black'],
) 

fig_117_006 = Figure(
      title='',
       marks=[lines_117_006, scatter_117_006],
       axes=[axis_x_006, axis_y_006],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=50, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )

block_4.children = [fig_117_005, fig_117_006]

In [19]:
################################
######5TH BLOCK#################
################################

block_5 = widgets.HBox([], layout=widgets.Layout(align_items='center'))

scale_y_008 = bqs.LinearScale(min = min(T_values), max = max(T_values), reverse = True)
          
axis_y_008 = bqa.Axis(
                scale=scale_y_008,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(min(T_values), max(T_values), 4),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='T',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px',
                )

lines_117_008 = bqm.Lines(
       x = trans_pressures,
       y = T_values, 
       scales = {'x': scale_x_006, 'y': scale_y_008}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = ['red'],
       #labels = T_values_str,
       )

scatter_117_008 = bqm.Scatter(
    name = '',
    x = trans_pressures,
    y = T_values,
    scales = {'x': scale_x_006, 'y': scale_y_008}, 
    #opacities = [1.0],
    visible = True,
    colors = ['black'],
)   

fig_117_008 = Figure(
      title='',
       marks=[lines_117_008, scatter_117_008],
       axes=[axis_x_006, axis_y_008],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=50, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )

scale_y_007 = bqs.LinearScale(min = min(T_values), max = max(T_values))
          
axis_y_007 = bqa.Axis(
                scale=scale_y_007,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(min(T_values), max(T_values), 4),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='T',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px',
                )

lines_117_007 = bqm.Lines(
       x = trans_pressures,
       y = T_values, 
       scales = {'x': scale_x_006, 'y': scale_y_007}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = ['red'],
       #labels = T_values_str,
       )

scatter_117_007 = bqm.Scatter(
    name = '',
    x = trans_pressures,
    y = T_values,
    scales = {'x': scale_x_006, 'y': scale_y_007}, 
    #opacities = [1.0],
    visible = True,
    colors = ['black'],
)   

fig_117_007 = Figure(
      title='',
       marks=[lines_117_007, scatter_117_007],
       axes=[axis_x_006, axis_y_007],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=50, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )

block_5.children = [fig_117_007, fig_117_008]

In [20]:
###################################
#############MAIN BLOCK############
###################################

main_block = widgets.VBox([], layout=widgets.Layout(align_items='center'))
main_block.children = [block_1,
                       block_2,
                       block_3,
                       block_4,
                       block_5
                      ]

main_block